# 第七章 虚拟机类加载机制

Java是纯粹的面向对象语言，因此类的加载就是其中十分重要的内容。一个类的生命周期包含以下七个方面的内容：

>类从被加载到虚拟机内存中开始，到卸载出内存为止，它的整个生命周期包括：加载（Loading），验证（Verification），准备（Preparation），解析（Resolution），初始化（Initialization），使用（Using）和卸载（Unloading）七个阶段。

注意：以下七个过程的执行并不是一个执行完毕后另一个才开始。他们的启动顺序是以以下顺序的。但是各个步骤的执行可能会有所重叠。

## 类加载的过程

* ### 加载

类的加载阶段是整个类生命周期的第一步，不过需要分清一个易混淆的概念：“加载”和“类加载”是两个不同的东西。前者指一个特定的步骤，而后者指的是整个将类加载进内存的动作。“类加载”包含了“加载”操作。在加载阶段，虚拟机需要完成以下三件事情：

1. 通过一个全限定名来获取定义此类的二进制字节流。

2. 将这个字节流所代表的静态存储结构转化为方法区运行时的数据结构。

3. 在内存中生成一个`java.lang.Class`对象，作为方法区的各种数据的访问入口。

如果是非数组类型，则加载过程就如上述，如果是数组类型则加载过程是针对数组中的组件类型而言的。

* ### 验证

验证过程主要为了确保`Class`文件的字节流中包含的信息符合虚拟机的要求，并且不会危害虚拟机自身的安全，主要包含以下几个部分的验证：

1. 文件格式验证：主要看`Class`文件的各个部分是否符合Java的要求，例如文件开头是否以`0xCAFEBABE`开头，常量池中是否有不符合要求的常量出现。

2. 元数据验证：主要验证该类是否有父类， 类中的字段是否与父类产生了矛盾等等。

3. 字节码验证：操作数栈中的类型和具体的操作所涉及的类型是否匹配等。

* ### 准备

准备阶段是把类变量（也就是静态变量）在方法区中分配内存的过程，需要注意的是类中的实例变量并不在这一步分配内存。分配内存后虚拟机还会对其进行0值化处理。但是如果一个类的静态字段在`Class`文件中的属性有`ConstantValue`属性的话，在这一步也会将其初始化为设定的值。

* ### 解析

解析阶段就是讲`Class`文件中的符号引用转化为直接引用的过程。所谓符号引用指的是类似于`CONSTANT_Class_info`这样的使用字符串形式产生的引用，而直接引用就是内存中指向对象的指针。

* ### 初始化

类的初始化过程并不是调用类构造方法初始化类的过程，该过程通过一个`<cinit>`方法来完成。这个方法中用来设定类中静态字段的的设定值，例如：

```Java
private static String name = "David";
```
这样的静态设定就会在初始化过程中完成。另外，类中的静态代码块也会在初始化过程中执行：

```Java
static{
    int a = 235;
}
```
总结来说：初始化过程就是执行类中的静态变量值设定以及静态代码块中内容的过程。

## 类加载器

类加载器是Java中进行类加载操作的程序。在Java中负责将类从`class`文件中加载到内存的操作。除此之外，类加载器还是标识某一个类的重要标志：一个类和另一个类进行类似于`equals`比较时通常是首先比较其类加载器是否相同，然后才比较类中的内容。如果两个类的类加载器都不同则接下来的比较就失去意义。

* ### 双亲委派模型

双亲委派模型是Java中实现类加载的一个模型。在双亲委派模型中，一共有三种类型的类加载器：

1. 启动类加载器（Bootstrap ClassLoader）：这种类加载器用于加载`<JAVA_HOME>/lib`下的类，使用C++编写。

2. 扩展类加载器（Extension ClassLoader）：这种类加载器用于加载虚拟机的扩展组件，主要用于加载`<JAVA_HOME>/lib/ext`下的类。

3. 应用程序类加载器（Application ClassLoader）：这种类加载器就是通常加载对象使用的，程序员自定义类加载器也是通过继承应用程序类加载器实现的。由于应用程序类加载器是`getSystemClassLoader()`方法的返回值，因此应用程序类加载器也被叫做系统类加载器。

双亲委派模型是说：除了顶层的启动类加载器之外，每一个类加载器都有父类加载器，例如扩展类加载器的父类是启动类加载器，而应用程序类加载器的父类又是扩展类加载器。当然这里的父类关系不是通过继承实现的，而是通过组合实现的。

当一个类需要被加载时，类加载器首先调用其父类加载器进行加载，直到启动类加载器。直到父类加载器不能加载此类才会在当前类加载器中加载。这样的特性就带来了一个好处：某一个类只能被某个类加载器加载。比如Java中`java.lang.*`的类都是由启动类加载器加载的。这样如果程序中如果用户又编写了`java.lang`开头的类，那么就会在启动类加载器中报错，否则程序中就会出现内容不同但名称相同的两个类。

以下是一个自定义类加载器的实例：

```Java
package com.ilmare.ClassLoader;

import java.io.IOException;
import java.io.InputStream;

public class HotClassLoader extends ClassLoader{
		
	public HotClassLoader() {
        /*
         * 这里传入一个null表示该类加载器的“父类”是启动类加载器，这样普通的类就无法通过其父类加载，就能够使用本类进行加载了。
         * 否则可能会出现其父类加载器直接就将普通类加载的情况，这样的话就无法调用findClass。
         */
		super(null);
	}

	@Override
	protected Class<?> findClass(String name) throws ClassNotFoundException {
		String destName = name.substring(name.lastIndexOf(".") + 1) + ".class";
		InputStream in = getClass().getClassLoader().getResourceAsStream(destName);
		if (in == null) {
			throw new ClassNotFoundException();
		}
		try {
			byte[] buffer = new byte[in.available()];
			in.read(buffer);
			Class clazz = defineClass(name, buffer, 0, buffer.length);
			return clazz;
		} catch (IOException e) {
			e.printStackTrace();
		}
		return super.findClass(name);
	}
	
}
```
用以下程序测试：

```Java
@Test
public void test2() {
    try {
    	String name = "com.ilmare.Domain.Person";
        HotClassLoader loader = new HotClassLoader();
        Class clazz = loader.loadClass(name);
        Constructor con = clazz.getConstructor(String.class, int.class, String.class, String.class);
        Object obj = con.newInstance("David", 25, "male", "China");
        System.out.println(obj.getClass());
        System.out.println(obj instanceof com.ilmare.Domain.Person);
        Class clazz1 = Class.forName(name);
        System.out.println(clazz1.equals(clazz));
        System.out.println(clazz1.getClassLoader() + " : " + clazz.getClassLoader());
    }catch(Exception e) {
    	e.printStackTrace();
    }
    	
}
```

输出以下内容：

```shell
class com.ilmare.Domain.Person
false
false
jdk.internal.loader.ClassLoaders$AppClassLoader@14514713 : com.ilmare.ClassLoader.HotClassLoader@5e955596
```
可以看到：即使是同一个类，如果使用不同的类加载器加载，也不会被认为是相同的。

最后，双亲委派模型的提出是比较晚的，且有的时候父类装载器也需要调用一些底层的代码。处于兼容的原因也会出现违反双亲委派的情况。

# 第八章 虚拟机字节码执行引擎

本节主要讲Java中方法的执行过程，具体的论述就是将Java虚拟机栈的组成。Java虚拟机栈是与Java类方法密切相关的数据结构，其中的组成部分“栈帧”与方法密切相关，每一个栈帧代表了一个方法执行的全部信息，包括局部变量表，操作数栈，方法连接以及返回值地址。

## Java虚拟机栈栈帧结构

Java虚拟机栈与Java方法密切相关，每一个栈帧就代表了类中的一个方法，栈帧中记录了这个方法的全部信息与运行时的要求。一个栈帧中包含了如下几个部分：

* 局部变量表：该变量表存储方法的参数以及方法中使用到的局部变量两部分内容，其最小分配单位为`slot`，一个`slot`的大小不固定，但是Java虚拟机规范中明确说明一个`slot`可以装载下一个`bool`等基本数据类型，而两个`slot`可以装载下一个`double`或`long`。一个栈帧的局部变量表大小在字节码中的方法中的`max_local`字段已经规定了最大的大小，单位是`slot`，需要注意的是`max_local`的最小大小为1，这是因为每个方法其实都隐式传入了`this`关键字，`this`关键字的类型是`reference`，这在局部变量表中占用一个`slot`存储。

* 操作数栈：该栈用于存储Java指令，在方法内进行方法调用时的操作数。其内容不固定，根据指令的执行动态地入栈出栈，其单位也是`slot`。例如Java字节码指令`iadd`用于对两个数进行加和，需要两个操作数分别存储在操作数栈的栈顶和次栈顶，当计算时就对着两个元素进行出栈，计算结果在存储回局部变量表。需要说明的是栈帧中的最大操作数栈深度也是在字节码文件中由方法属性的`max_stack`属性确定的，程序运行中操作数栈的最大深度不会超过`max_stack`。

* 动态连接：在字节码文件中的方法区域中存储着类中的所有方法，方法中可能还存在对其他方法的调用。这些方法和方法调用在字节码中是以“符号引用”的形式存在的，所谓“符号引用”就是使用字符的形式表现引用关系。在程序运行时需要将这些引用关系转化成“直接引用”，也就是使用指针的形式确定方法的执行入口。
  1. 这部分的内容有的在代码编译的时候就确定了具体要执行的方法版本（考虑到Java的面向对象性，同一个方法签名可能有多个版本，也就是多个重载或重写版本），然后在程序解析的时候将方法的入口地址确定，这部分方法主要指的是“非虚方法”也就是类的`private`方法，静态方法等。因为这些方法无法被重载或重写，因此可以在运行前就唯一确定。
  2. 另一部分方法可能有多个版本，这就无法仅仅通过解析来完全确定了。这部分方法可以通过分派的形式来去确定。分派与解析不是二选一的关系，而是在不同层次上对方法进行筛选。这种筛选也可以在程序运行前也可以是在程序运行时，具体可以分为静态分派和动态分派。根据分派宗量的种类可以分为单分派和多分派。
  
* 返回值地址：通常一个方法会有返回值，这个返回值可能是其他调用方法某一个操作的操作数地址，因此栈帧的这个字段就指明了这些信息。

## 分派

分派其实也是为了解决调用哪一个方法的问题，一些非虚方法的调用可以通过字节码来唯一确定进而在类的解析阶段就完成从符号引用到直接引用的改变。但是对于一些非虚方法就需要进一步的筛选。现在举例来说明：

### 静态分派

所谓静态分派就是根据类的静态类型或者外观类型进行分派，假如有以下代码：

```Java
class super_super{}
class super extends super_super{}
class sub extends super{}
class App{
    public void func(super_super obj){
        System.out.println("This is super_super");
    }
    public void func(super obj){
        System.out.println("This is super");
    }
    public void func(sub obj){
        System.out.println("This is sub");
    }
    public static void main(String[] args){
        super_super obj1 = new super();
        super_super obj2 = new sub();
        func(obj1);
        func(obj2);
    }
}
```
对于上面的程序可以明显看出：`obj1`和`obj2`的实际类型分别是`super`和`sub`，但是它们的静态类型都是`super_super`，那么在如上代码中`App`类对方法`func`进行了三次重载，当使用者实际调用时就会产生如下输出：

```shell
This is super_super
This is super_super
```
可见：虚拟机在进行方法的分派时是根据对象的静态类型进行方法分派的。那么，使用类的静态类型完成的方法分派就是静态分派。

### 动态分派

所谓动态分派就是根据类的实际类型进行的分派，这样的分派通常在运行时根据对象的实际类型来确定方法版本的调用。例如有如下代码：

```Java
class super_super{
    public void func(){
        System.out.println("This is super_super");
    }
}
class super extends super_super{
    public void func(){
        System.out.println("This is super");
    }
}
class sub extends super{
    public void func(){
        System.out.println("This is sub");
    }
}
class App{ 
    public static void main(String[] args){
        super_super obj1 = new super();
        super_super obj2 = new sub();
        obj1.func();
        obj2.func();
    }
}
```
这样的代码在使用时将会在`main`方法内部生成`invokevirtual`的方法调用指定，该指令的操作数之一就是调用其的对象，该对象叫做方法的“接收者”，是被当做操作数存储在操作数栈中的，其类型与静态类型无关是由其实际类型决定的。之所以叫做动态分配是因为从字节码上看`obj1.func()`调用都是`invokevirtual`到`func`。但是跟静态分派不同的是，具体调用哪一个版本的`func`是由操作数栈中的操作数决定的，也就是调用对象的实际类型，而这个操作数只有在程序运行时才唯一确定，这也就是“动态”二字的来源。

上面程序的输出为：

```shell
This is super
This is sub
```

### 单分派与多分派

介绍单分派和多分派之前首先要明确宗量的概念。所谓宗量就是一个方法的调用者（接收者）以及方法的参数。

* 单分派：如果一个方法的分派使用一个宗量（也就是使用方法的接收者或使用方法的参数）就能唯一确定方法的调用版本，那么就说这样的分派是单分派。例如的动态分派例子：程序只使用`func`方法的接收者就唯一确定，这就是动态单分派。事实上，哪怕`func`方法存在参数，这也是一个动态单分派，这是由于动态分派是在程序运行期间进行的，是Java方法动态性的一种表现，而如果一个方法进行动态分派那么其方法参数在编译时就已经完全确定在运行时并不会参与到分派决策过程，因此其是否具有参数都不会影响其分派的宗量，动态分派就是单分派。

* 多分派：这个概念和单分派相对，也就是如果一个方法的分派需要用到多余一个宗量来确定就是多分派。上面单分派的例子中：`func`方法的接收者是`App`，而参数可以有三种，这样确定`func`版本就需要方法的参数和接收者两个宗量，那么这就是一个静态多分派的例子。

事实上，Java是一种静态多分派，动态单分派的语言。

## Java语言的动态性

通常我们说Java是一中静态类型的语言，这不仅仅体现在Java中每一个标识符都有一个具体的类型，更体现在Java无法向C++一样传入一个函数指针。这完全是因为Java在执行方法时使用的`invoke*`字节码指令，这种字节码指令后面会跟一个常量索引类型是`CONSTANT_Methodref_info`，这种常量就把方法的所在类和方法描述符完全确定了，这样的特性使得Java在编译阶段就杜绝了Java的动态性。

Java的动态性是使用了新的字节码指令`invokedynamic`，这种新的字节码指令不再索引到一个特定的方法上，而是索引到一个`Bootstrap method`上，再由这个方法来确定调用的方法。

# 第十章 早期（编译期）优化

Java中编译可以有多种解释，可以指将`*.java`编译成`*.class`的过程，也可以指将字节码文件转成机器码的过程。但是在本节中编译仅仅值将`*.java`编译成`*.class`文件的过程。这个过程其实并没有什么可以优化的，因此本节主要是讲`javac`这个编译器如何将一个`*.java`文件翻译成`*.class`文件。`javac`进行编译的时候将遵循以下步骤：

* 解析与填充符号表的过程。

* 插入式注解处理器的注解处理过程。

* 分析与字节码生成过程。

## 解析与填充符号表的过程

这个过程可以分为解析过程和符号表填充过程，解析过程指的是源程序文件的词法分析和语法分析的过程。

* 词法分析：词法分析指的是将源程序文件中的字符转换成标记的过程，所谓标记指的就是类似于`int`这样的类型，`a`这样的标识符或者`165`这样的字面量等。这是Java语言中最小的处理单元。

* 语法分析：语法分析就是形成抽象语法树（AST）的过程，语法树的每一个节点都是一个基本的语法结构，例如包，类型，修饰符等。形成语法树之后编译器就不会针对源代码文件进行操作了，之后的操作都是针对语法树进行的。

完成了词法分析和语法分析之后就需要进行符号表填充。符号表指的是一组由符号地址和符号信息内容形成的表格，可以由哈希表形成，也可以由其他的数据结构形成。符号表中的信息在编译的不同阶段都会用到，比如说类的默认构造函数就会在符号表中定义。

## 插入式注解处理器的注解处理过程

之所以是“插入式注解处理器”是因为这些注解处理器可以在源代码编译时插入式地影响程序编译的过程。注解处理器在处理注解时可以改变抽象语法树的结构，因此这个阶段和“解析与符号填充阶段”是交替进行的，每次完成注解处理后（如果确实改变了语法树的结构）就会再进行一次解析与符号表填充以更新语法树结构。

## 分析与字节码生成

这个阶段是整个编译过程中非常重要的，这个阶段的全名应该是“语义分析与字节码生成”，主要有以下几个阶段：

* 标注阶段：在这个阶段中将会检查语法树中使用的变量是否事先声明过，变量的赋值类型是否匹配。以及还会出现“折叠”操作，该操作会简化程序，例如有这样的赋值操作`int a = 1 + 3`，折叠操作就会将`1 + 3`转化为`4`。

* 控制和数据流分析：这一步我的理解是针对程序的逻辑进行检查。例如检查变量在使用前是否赋值，如果一个方法拥有返回值那么方法的每一条执行路径上是否都会返回相应的值。

* 语法糖处理：Java作为一种低糖语言在这一步并没有什么要做的，但是这一步在泛型的处理上有很重要的作用，这在后面会讲到。

* 字节码生成：这一步仅仅是将语法树和符号表转化成字节码写在文件中，并伴随有少量的代码添加和转换工作。最明显的一个代码添加和转换工作是将类的构造方法`<clinit>()`和实例构造方法`<init>()`加入到语法树中，同时会将`static{}`代码块和`{}`代码块中的代码自动添加到`<clinit>()`和`<init>()`方法中。

## Java中的泛型

泛型有两类，一种是真泛型，一种是伪泛型。泛型最初在C++中的模板中就已经有迹可循。Java中的泛型主要起到减少程序员编码错误的功能，首先要明确的是Java中的泛型是一种伪泛型，与之相对的，C#等语言中的泛型是一种真泛型。

所谓真泛型就是`List<Integer>`和`List<String>`是两个完全不同的类，这一点可以在其源程序的编译代码中得到证实。

伪泛型则是另一种实现思路，由于Java中继承的存在，子类对象可以强制转化成父类对象，而Java中处于继承树顶端的只有一个`Object`对象，因此类似于`List`这样的集合类在`get`方法中都返回的是一个`Object`对象，实际上是将一个特定对象强制转化成`Object`对象返回的。当程序员在程序中使用泛型（例如`List<Integer>`）并编译后可以发现：泛型消失了，程序文件中的泛型并不会在字节码中有任何痕迹，这样的特性被叫做类型擦除。字节码唯一做的就是在相关方法返回时进行了一次类型强制转换。由于这样的特性，像下面的代码是没办法编译通过的：

```Java
public class App{
    public void func(List<String> list){
        System.out.println("This is void func(List<String> list)");
        return;
    }
    public void func(List<Integer> list){
        System.out.println("This is void func(List<Integer> list)");
        return;
    }
}
```
编译不通过的原因也很简单：由于类型擦除的缘故，这两个方法编译后的特征标将完全一致（泛型将会被消除），而字节码文件中不可能允许出现两个完全一样的函数签名。当然，如果更改这两个方法的返回值，这个程序就能够被编译通过：

```Java
public class App{
    public String func(List<String> list){
        System.out.println("This is void func(List<String> list)");
        return list.get(0);
    }
    public Integer func(List<Integer> list){
        System.out.println("This is void func(List<Integer> list)");
        return list.get(0);
    }
}
```
这可能让人有些疑惑，不是说Java中方法的重载只和方法的特征标有关吗，为什么更改了方法的返回值也可以呢？这是因为在字节码中只要是两个方法的签名不一样那么这两个方法就可以共存，而方法签名中包含方法的返回值。当然，这样的代码可以通过编译但是无法执行。

# 第十一章 晚期（运行期）优化

 晚期优化主要指的是在运行期间的优化，这主要是针对JIT编译器而言的，所谓JIT编译器是`Just In Time Compiler`的缩写。Java由于本身是一种解释性语言，因此在执行速度上可能会比较慢，为了解决这个问题，Java对某些需要多次执行的代码（热点代码）会进行运行时编译，JIT编译器会将这部分代码编译成本地机器码以提高运行速度。晚期优化就是针对运行时编译提出的。当然，运行时编译并不是虚拟机规范的一部分，并没有强制要求实现。
 
Java中拥有编译器和解释器，这二者通常是配合工作的。除了之前提到的编译器会编译热点代码以提高执行速度外，解释器也会充当编译器的“逃生门”，这是由于编译器有时为了优化代码可能会采取一些激进措施，但是当这些激进措施失效时就需要将代码返回给解释器执行，这虽然会导致执行效率下降，但是能保证执行的正确性，这就是逃生门的作用。

编译器有两个类型，分别是客户端编译器（C1编译器）和服务端编译器（C2编译器），这两种编译器的区别主要在于客户端编译器的优化措施较为浅层，代价也相对较小。但是服务端编译器的优化是全局的，也需要更多的时间。通常情况下，运行时编译需要一定的时间，这会影响一些程序执行的效率，因此JIT编译器会分层执行代码编译：

* 第0层：程序解释执行，解释器不开启性能监控功能，可触发第一层编译。

* 第1层：也称为C1编译，将字节码编译成本地机器码，进行简单有效的优化，如有必要会加入一些监控逻辑。

* 第2层：称为C2编译，将字节码编译成本地机器码，会使用一些耗时较长的优化措施，甚至会根据性能佳监控数据进行一些激进的优化措施。

## 运行时编译的触发条件

通常情况下，运行时编译器会将执行次数较多的代码编译成本地代码以提高执行速度。那么，什么样的代码算是执行频率较高的代码呢？（需要说明的是：运行时编译都是针对整个方法而言的）：

* 基于采样的热点探测：这种方法不是很常用，该方法会周期性检查Java虚拟机栈，如果某一个方法多次出现在栈顶，那么就会针对该方法进行运行时编译。

* 基于计数器的热点探测：这种方法应用比较广泛，通常是设置计数器对某一个代码块的执行频率进行统计，达到一定的阈值之后就会触发本地编译。

Hotpot虚拟机采用的就是基于计数器的热点探测，其设置了两种类型的计数器：

* 方法调用计数器：顾名思义，该计数器就是针对方法调用次数的计数器，每次调用就会将计数器的数值加1，当时如果经过一段时间该计数器的值还没有达到事先设置的阈值，该计数器的值就会衰减一般，这个过程叫做“热度衰减”，而这个时间被称为“半衰周期”。当调用计数器的值和回边计数器的值加和达到事先设置的阈值时，就会启动运行时编译，这个编译过程是在后台独立线程中进行的，编译完成之前还是将方法解释执行。

* 回边计数器：该计数器是针对方法执行的另一种情况设计的，如果某一个方法中有一个循环被多次执行，那么即使这个方法不会被多次调用，那么这个方法也被称为是热点代码。回边的意思就是代码返回刚才执行的地方，这就是针对循环而言的。每执行一次回边指令（也就是每循环一次循环体）都会将回边计数器的值加1，当回边计数器和方法调用计数器的值之和达到设定阈值之后就会触发运行时编译，由于由循环体触发的编译是在方法执行过程中，且编译好之后会直接执行编译版本，因此这种编译又被称为是“栈上替换”，即`OSR(On Stack Replacement)`。由于编译会有一定的耗时，因此在触发编译之后会将回边计数器的值减少，以防多次触发编译。

## 编译优化

编译优化的手段非常多，通常情况下，Java的运行时编译器首先会针对代码进行方法内联，这是针对代码的最基础优化，也是其他优化手段的前提。在这一节中主要介绍四种优化手段：

* 语言无关的经典优化技术——公共子表达式消除：所谓公共子表达式就指的是：如果一个表达式E的值已经被计算过了，并且从先前的计算到现在E的值都没有发生变化，那么E的这次出现就成为了公共子表达式。例如：

```Java
int d = (c * b) * 12 + a + (a + b * c)
```
    那么，b*c就是公共子表达式。编译器在计算完成b*c的值之后就会将所有的b*c都替换成计算值，以避免重复计算带来的性能损耗。

* 语言相关的经典优化技术——数组范围检查消除：这是针对Java语言自身特性的。由于Java异常机制，保证了他能够对诸如数组越界的情况进行处理，但是这种处理本身也是需要消耗性能的。因此这种优化就是为了消除这样的损耗：例如当一个数组采用随机访问`foo[3]`，那么在编译器中只需要看这个随机访问的索引是否超出数组的大小，这样就不需要在运行时动态判断，如果是在循环中，就判断循环的上下界是否处于数组的大小范围内就行了。

* 最重要的优化技术——内联优化：这种优化措施在Java中比较难以实现，这是因为Java语言的动态性，当类与类之间拥有继承关系时，具体调用哪一个版本的方法只有在执行时才能够完全确定（又方法的接收者确定），只有那些非虚方法（静态方法，私有方法等无法继承的方法）才能够在编译时确定方法的具体版本，进而实现内联。为了解决这个问题，Java虚拟机团队使用了一种称为`CHA(Class Hierarchy Analysis)`“类型继承关系分析”技术来确定某一个类的继承关系以及方法的调用版本，一次来实现虚方法的内联。但是这是一种激进的优化措施，因此要使用解释器来作为“逃生门”保证程序的执行正确性。

* 最前沿的优化技术——逃逸分析：这种分析主要是看一个方法中的变量会不会被其他方法使用（例如作为参数传递到其他方法中）或被其他线程使用，如果某一个变量是不逃逸的，那么就能够有很多优化手段，例如将这个变量的内存在Java虚拟机栈中分配，那么该变量的垃圾回收就会变得简单，能够大大节省内存，这种有话叫做“栈上分配”；如果一个变量只被一个线程使用，那么就不必对这个变量加锁，减少了加锁开销。逃逸分析还是一种非常前沿的优化措施，也是编译器优化的一个重要方向。